In [ ]:
import math

import numpy as np
import pandas as pd
import spacy


from collections import Counter
from pathlib import Path
from pprint import pprint

In [ ]:
dis = ['parser', 'ner']
# print(nlp.pipe_names)

In [ ]:
%%time
file = Path('input', 'enron_mails.p')
df = pd.read_pickle(file)

In [ ]:
df['length'] = df['text'].apply(len)
Q1 = df['length'].quantile(0.25)
Q3 = df['length'].quantile(0.75)
IQR = Q3 - Q1
# df.loc[df['length'] > (Q3 + 15 * IQR),:]
df.sort_values(by=['length'], ascending=False).head(10)

In [ ]:
extreme_vals = [132160, 480942, 337694]

In [ ]:
df.drop(extreme_vals, inplace=True)

In [ ]:
def chunkit(df, chunk_size = 50000): 
    num_chunks = math.ceil(len(df) / chunk_size)
    if len(df) % chunk_size != 0:
        num_chunks += 1
    for i in range(num_chunks):
        yield df[i*chunk_size:(i + 1) * chunk_size].copy()

In [ ]:
dfch = chunkit(df)

In [ ]:
# def get_tokens(text):
#     doc = nlp(text)
#     res = [token.text for token in doc]
    
#     return res

# def get_token_lemma(text):
#     doc = nlp(text)
#     res = [(token, token.lemma_) for token in doc]
    
#     return res


# def get_token_lemma_pos(text):
#     doc = nlp(text)
#     res = [(token, token.lemma_, token.pos_) for token in doc]
    
#     return res


def get_tokens(doc):
    res = [token for token in doc]
    
    return res


def get_token_lemma(doc):
    res = [(token, token.lemma_) for token in doc]
    
    return res


def get_token_lemma_pos(doc):
    res = [(str(token.text), str(token.lemma_), str(token.pos_)) for token in doc]
    
    return res

In [ ]:
%%time
for i, chunk in enumerate(dfch):
    nlp = spacy.load('en_core_web_sm', disable=dis)
    docs = nlp.pipe(chunk['text'].tolist(), n_process=8)
    chunk['doc'] = [doc for doc in nlp.pipe(chunk["text"].tolist())]
    chunk.loc[:,['id', 'doc']].to_pickle(Path('output', f'email_spacy_{i}_doc.pkl'))

In [ ]:
%%time
dfch = chunkit(df)
for i, chunk in enumerate(dfch):
    nlp = spacy.load('en_core_web_sm', disable=dis)
    docs = nlp.pipe(chunk['text'].tolist(), n_process=8)
    chunk['tlp'] = [get_token_lemma_pos(doc) for doc in nlp.pipe(chunk["text"].tolist())]
    chunk.loc[:,['id', 'tlp']].to_pickle(Path('output', f'email_spacy_{i}_tlp.pkl'))

In [ ]:
%%time
df['tlp'] = df['text'].apply(get_token_lemma_pos)

In [ ]:
text1 = df.loc[0,'text']

In [ ]:
doc = nlp(text1)

In [ ]:
for ent in doc:
    print((ent.text, ent.lemma_, ent.pos_))

In [ ]:
# nlp.max_length = 1650000

In [ ]:
b_s = 500
n_p = 6

docs = nlp.pipe(df['text'].tolist(), batch_size=b_s, n_process=n_p, disable=dis)

In [ ]:
%%time
df["spacy"] = [doc for doc in docs]

In [ ]:
%%time
tok_list = []
for doc in docs:
    tok_list.append(get_tokens(doc))

In [ ]:
len(tok_list)

In [ ]:
%%time

t_l_list = []
for doc in docs:
    t_l_list.append(get_token_lemma(doc))

In [ ]:
%%time
tlp_list = []
for doc in docs:
    tlp_list.append(get_token_lemma_pos(doc))

In [ ]:
texts = df.loc[:2,'text']
texts

In [ ]:
docs = nlp.pipe(texts)

In [ ]:
for token in next(docs):
    print(dir(token))

In [ ]:
for token in next(docs):
    print((token, token.lemma_, token.pos_))

In [ ]:
for doc in docs:
    print(dir(doc))

In [ ]:
for doc in docs:
    print(doc.is_nered)

In [ ]:
text = df.loc[0,'text']

In [ ]:
text

In [ ]:
from dframcy import DframCy
dframcy = DframCy(nlp)

In [ ]:
doc = dframcy.nlp(u'Apple is looking at buying U.K. startup for $1 billion')
annotation_dataframe = dframcy.to_dataframe(doc)
annotation_dataframe